# This code is an interactive way to compare the boroughs and works as a recommendation system

In [2]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

#We define the paths to our CSV files for each year
csv_files = {
    "20-21": r'../data/pas_data_ward_level/PAS_ward_level_FY_20_21.csv',
    "19-20": r'../data/pas_data_ward_level/PAS_ward_level_FY_19_20.csv',
    "18-19": r'../data/pas_data_ward_level/PAS_ward_level_FY_18_19.csv',
    "17-18": r'../data/pas_data_ward_level/PAS_ward_level_FY_17_18.csv',
    "15-17": r'../data/pas_data_ward_level/PAS_ward_level_FY_15_17.csv'
}

#We define here the borough column name for each year
borough_column_mapping = {
    "20-21": "Borough",
    "19-20": "C2",
    "18-19": "C2",
    "17-18": "C2",
    "15-17": "C2"
}

#We define here the ethnic group column name for each year
ethnic_group_column_mapping = {
    "20-21": "ReNQ147",
    "19-20": "NQ147r",
    "18-19": "NQ147r",
    "17-18": "NQ147r",
    "15-17": "NQ147r"
}

#Define the questions and their corresponding statement and values (for the dashboard also)
questions = {
    "Q13": {
        "statement": "Q13: To what extent are you worried about crime in this area?",
        "values": ["Very worried", "Fairly worried"]
    },
    "Q15": {
        "statement": "Q15: To what extent are you worried about anti-social behaviour in this area?",
        "values": ["Very worried", "Fairly worried"]
    },
    "Q60": {
        "statement": "Q60: Taking everything into account, how good a job do you think the police IN YOUR AREA are doing?",
        "values": ["Poor", "Very poor"]
    },
    "Q62A": {
        "statement": "Q62A: To what extent do you agree with these statements about the police in your area?\
        By 'your area' I mean within 15 minutes' walk from your home. \
        They can be relied on to be there when you need them",
        "values": ["Tend to disagree", "Strongly disagree"]
    },
    "Q62B": {
        "statement": "Q62B: To what extent do you agree with these statements about the police in your area?\
        By 'your area' I mean within 15 minutes' walk from your home. \
        They would treat you with respect if you had contact with them for any reason.",
        "values": ["Tend to disagree", "Strongly disagree"]
    },
    "Q62C": {
        "statement": "Q62C: To what extent do you agree with these statements about the police in your area?\
        By 'your area' I mean within 15 minutes' walk from your home. \
        The police in your area treat everyone fairly regardless of who they are.",
        "values": ["Tend to disagree", "Strongly disagree"]
    },
    "Q62D": {
        "statement": "Q62D: To what extent do you agree with these statements about the police in this area?\
        By 'this area' I mean within 15 minutes' walk from here. They can be relied on to deal with minor crimes",
        "values": ["Tend to disagree", "Strongly disagree"]
    },
    "Q62E": {
        "statement": "Q62E: To what extent do you agree with these statements about the police in this area?\
        By 'this area' I mean within 15 minutes' walk from here. They understand the issues that affect this community",
        "values": ["Tend to disagree", "Strongly disagree"]
    },
    "Q62TG": {
        "statement": "Q62TG: To what extent do you agree with these statements about the police in your area? \
        By 'your area' I mean within 15 minutes' walk from your home. \
        The police in your area listen to the concerns of local people.",
        "values": ["Tend to disagree","Strongly disagree"]
    },
    "A121": {
        "statement": "A121: How confident are you that the Police in your area use their stop and search powers fairly?",
        "values": ["Not very confident", "Not at all confident"]
    }, 
    "Q62F": {
        "statement": "Q62F: To what extent do you agree with these statements about the police in your area?\
        By 'your area' I mean within 15 minutes' walk from your home.\
        They are dealing with the things that matter to people in this community",
        "values": ["Tend to disagree","Strongly disagree"]
    },
    "Q62H": {
        "statement": "Q62H: To what extent do you agree with these statements about the police in this area?\
        By 'this area' I mean within 15 minutes' walk from here. The police in this area are helpful",
        "values": ["Tend to disagree","Strongly disagree"]
    },
    "Q62TI": {
        "statement": "Q62TI: To what extent do you agree with these statements about the police in this area?\
        By 'this area' I mean within 15 minutes' walk from here. The police in this area are friendly and approachable",
        "values": ["Tend to disagree","Strongly disagree"]
    },
    "Q62TJ": {
        "statement": "Q62TJ: To what extent do you agree with these statements about the police in this area?\
        By 'this area' I mean within 15 minutes' walk from here. The police in this area are easy to contact",
        "values": ["Tend to disagree", "strongly disagree"]
    },
    "NQ135BD": {
        "statement": "NQ135BD: To what extent do you agree or disagree with the following statements:\
        The Metropolitan Police Service is an organisation that I can trust",
        "values": ["Tend to disagree", "strongly disagree"]
    },
    "NQ135BH": {
        "statement": "NQ135BH: To what extent do you agree or disagree that\
        the police in your local area are sufficiently held accountable for their actions?",
        "values": ["Tend to disagree", "strongly disagree"]
    }
}

#A function to calculate percentages for each ethnic group and borough
def calculate_percentages(df, question, values, ethnic_group, borough_column, ethnic_column):
    if question not in df.columns:
        return pd.DataFrame(columns=[borough_column, ethnic_column, 'Percentage'])
    
    if ethnic_group != 'All':
        df = df[df[ethnic_column] == ethnic_group]
    
    #We filter here the dataframe based on the question values
    filtered_df = df[df[question].isin(values)]
    
    #This is to count the total responses and filtered responses for each ethnic group within each borough
    total_counts = df.groupby([borough_column, ethnic_column]).size().rename('Total')
    disagree_counts = filtered_df.groupby([borough_column, ethnic_column]).size().rename('Disagree')
    
    #Combine and calculate percentages
    combined_counts = pd.concat([total_counts, disagree_counts], axis=1).fillna(0)
    combined_counts['Percentage'] = (combined_counts['Disagree'] / combined_counts['Total']) * 100
    combined_counts['Percentage'] = combined_counts['Percentage'].round(2)
    combined_counts = combined_counts.reset_index()
    
    return combined_counts[[borough_column, ethnic_column, 'Percentage']]

#Initialize the Dash app (with its layout and features)
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Borough Recommendation System"),
    
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in csv_files.keys()],
        value='20-21'  #We set the default value to the latest year
    ),
    
    dcc.Dropdown(
        id='question-dropdown',
        options=[{'label': questions[q]['statement'], 'value': q} for q in questions],
        value='Q13'
    ),
    
    dcc.Dropdown(
        id='borough-dropdown',
        value='All'  #We set the default value to 'All'
    ),
    
    dcc.Dropdown(
        id='ethnicity-dropdown',
        value='All'
    ),
    
    dcc.RadioItems(
        id='top-bottom-radio',
        options=[
            {'label': 'Top 5 Percentages', 'value': 'top'},
            {'label': 'Bottom 5 Percentages', 'value': 'bottom'}
        ],
        value='top',
        labelStyle={'display': 'inline-block', 'marginRight': '10px'}
    ),
    
    dcc.Graph(id='percentage-graph'),
    
    html.Div(id='advice-text', style={'marginTop': '20px', 'fontSize': '18px'}),
    html.Div(id='rank-text', style={'marginTop': '10px', 'fontSize': '18px'})
])

@app.callback(
    [Output('percentage-graph', 'figure'),
     Output('advice-text', 'children'),
     Output('rank-text', 'children'),
     Output('borough-dropdown', 'options'),
     Output('ethnicity-dropdown', 'options'),
     Output('ethnicity-dropdown', 'value')],
    [Input('year-dropdown', 'value'),
     Input('question-dropdown', 'value'),
     Input('borough-dropdown', 'value'),
     Input('ethnicity-dropdown', 'value'),
     Input('top-bottom-radio', 'value')]
)
def update_graph(selected_year, selected_question, selected_borough, selected_ethnicity, top_bottom):
    #We load the data for the selected year
    df = pd.read_csv(csv_files[selected_year])
    
    #Rename the borough and ethnic group columns if necessary
    borough_column = borough_column_mapping[selected_year]
    ethnic_column = ethnic_group_column_mapping[selected_year]
    if borough_column != 'Borough':
        df = df.rename(columns={borough_column: 'Borough'})
    if ethnic_column != 'ReNQ147':
        df = df.rename(columns={ethnic_column: 'ReNQ147'})
    
    #Calculate percentages based on selected question, borough, and ethnicity
    percentages_df = calculate_percentages(df, selected_question, questions[selected_question]['values'], selected_ethnicity, 'Borough', 'ReNQ147')
    
    if percentages_df.empty:
        return {}, "Question not available for the selected year.", "", [], [], 'All'

    if selected_borough == 'All':
        #Filter the percentages for all boroughs
        borough_percentages = percentages_df.sort_values(by='Percentage')
    else:
        #Filter the percentages for the selected borough
        borough_percentages = percentages_df[percentages_df['Borough'] == selected_borough].sort_values(by='Percentage')
    
    if selected_ethnicity != 'All':
        #Filter the percentages for the selected ethnicity
        borough_percentages = borough_percentages[borough_percentages['ReNQ147'] == selected_ethnicity]

    if selected_borough == 'All' and selected_ethnicity == 'All':
        #Show all boroughs and all ethnic groups
        fig = px.bar(borough_percentages, x='Borough', y='Percentage', color='ReNQ147', barmode='group', title=questions[selected_question]['statement'],
                     hover_data={'ReNQ147': False, 'Percentage': True}, labels={'ReNQ147': 'Ethnic group'})
    elif selected_borough == 'All':
        #Show all boroughs for a specific ethnic group
        fig = px.bar(borough_percentages, x='Borough', y='Percentage', title=questions[selected_question]['statement'],
                     hover_data={'ReNQ147': False, 'Percentage': True}, labels={'ReNQ147': 'Ethnic group'})
    else:
        if selected_ethnicity == 'All':
            fig = px.bar(borough_percentages, x='ReNQ147', y='Percentage', title=questions[selected_question]['statement'],
                         hover_data={'ReNQ147': False, 'Percentage': True}, labels={'ReNQ147': 'Ethnic group'})
        else:
            if top_bottom == 'top':
                top_boroughs = percentages_df[(percentages_df['ReNQ147'] == selected_ethnicity)].nsmallest(5, 'Percentage')
            else:
                top_boroughs = percentages_df[(percentages_df['ReNQ147'] == selected_ethnicity)].nlargest(5, 'Percentage')
            
            #Include the selected borough
            selected_borough_data = percentages_df[(percentages_df['Borough'] == selected_borough) & (percentages_df['ReNQ147'] == selected_ethnicity)]
            comparison_data = pd.concat([top_boroughs, selected_borough_data]).drop_duplicates().sort_values(by='Percentage')
            
            fig = px.bar(comparison_data, x='Borough', y='Percentage', title=questions[selected_question]['statement'],
                         hover_data={'ReNQ147': False, 'Percentage': True}, labels={'ReNQ147': 'Ethnic group'})
            
            #This is to hHighlight the selected borough
            fig.update_traces(marker_color=['red' if borough == selected_borough else '#636EFA' for borough in comparison_data['Borough']])
    
    # Generate advice and rank text
    advice, rank_text = "", ""
    
    if selected_borough != 'All' and selected_ethnicity != 'All':
        all_boroughs_sorted = percentages_df[percentages_df['ReNQ147'] == selected_ethnicity].sort_values(by='Percentage')
        selected_borough_rank = all_boroughs_sorted.reset_index().index[all_boroughs_sorted['Borough'] == selected_borough][0] + 1
        num_boroughs = len(all_boroughs_sorted)
        
        top_3_boroughs = all_boroughs_sorted.head(3)['Borough'].tolist()
        if selected_borough in top_3_boroughs:
            if selected_borough == top_3_boroughs[0]:
                advice = f"{selected_borough} is the best performing borough for the selected ethnic group."
            else:
                better_boroughs = [b for b in top_3_boroughs if b != selected_borough]
                advice = f"{selected_borough} is among the top 3 performing boroughs. Consider getting advice from {', '.join(better_boroughs)}."
        else:
            advice = f"{selected_borough} is not in the top 3 performing boroughs. Consider getting advice from {', '.join(top_3_boroughs)}."

        rank_text = f"{selected_borough} ranks {selected_borough_rank} out of {num_boroughs} boroughs for the selected ethnic group."

    #Update the options for the borough and ethnicity dropdowns based on the selected year
    borough_options = [{'label': 'All Boroughs', 'value': 'All'}] + [{'label': b, 'value': b} for b in df['Borough'].unique()]
    ethnicity_options = [{'label': 'All', 'value': 'All'}] + [{'label': eth, 'value': eth} for eth in df['ReNQ147'].dropna().unique()]
    
    #Handle dropdown value reset if current value is not in new options
    if selected_ethnicity not in [option['value'] for option in ethnicity_options]:
        selected_ethnicity = 'All'

    return fig, advice, rank_text, borough_options, ethnicity_options, selected_ethnicity

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)  #Change the port number if needed, due to port issues


---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
Cell In[2], line 201, in update_graph(
    selected_year='20-21',
    selected_question='Q13',
    selected_borough='All',
    selected_ethnicity='All',
    top_bottom='top'
)
    186 @app.callback(
    187     [Output('percentage-graph', 'figure'),
    188      Output('advice-text', 'children'),
   (...)
    199 def update_graph(selected_year, selected_question, selected_borough, selected_ethnicity, top_bottom):
    200     #We load the data for the selected year
--> 201     df = pd.read_csv(csv_files[selected_year])
        csv_files[selected_year] = 'data\\pas_data_ward_level\\PAS_ward_level_FY_20_21.csv'
        selected_year = '20-21'
        pd = <module 'pandas' from '/Users/alexraudvee/Desktop/TU_e/year_2/Q4_Y2_Data_Challenge_2/JBG050_Group_8/.venv/lib/python3.10/site-packages/pandas/__init__.py'>
        csv_files = {'20-21': '